In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.models import NumeralTickFormatter
from bokeh.io import output_notebook
from datetime import timedelta
import numpy as np
from IPython.core.display import display, HTML
from bokeh.models import HoverTool
from bokeh.layouts import row,column,grid
from bokeh.plotting import figure, show

In [2]:
display(HTML("<h1 align='center'>A Brief Overview of Macroeconomic Indicators Today</h1>"))

In [3]:
display(HTML("<h2 align='center'>Ryan Miller<br><br>05/13/2020</h2>"))

In [4]:
display(HTML("<h6>The loading symbol below starts the data visualization tool in the notebook.</h6>"))

In [5]:
output_notebook();

Loading BokehJS ...

In [6]:
from fredapi import Fred

with open(r"C:\Users\ryana\Desktop\fred_api_key.txt") as f:
    api_key = f.read()
fred = Fred(api_key=api_key)

#GDP Components
personal_consumption_expenditures = fred.get_series('PCEC')
gross_private_domestic_investment = fred.get_series('GPDI')
government_consumption_expenditures = fred.get_series('GCE')
net_exports = fred.get_series('NETEXP')

#GDP Measures
nominal_gdp = fred.get_series('GDP')
gdp_deflator = fred.get_series('GDPDEF')
real_gdp = fred.get_series('GDPC1')

#Money Supply
m2_money_supply = fred.get_series('M2')
m2_grouped_by_quarter = m2_money_supply.groupby(pd.Grouper(freq="Q")).last()
m2_grouped_by_quarter.index = m2_grouped_by_quarter.index + timedelta(days=1)

m2_money_velocity = fred.get_series('M2V')

#Unemployment Rate
unemployment_rate = fred.get_series('UNRATE')
unemployment_rate = unemployment_rate/100
unemployment_rate_quarterly = unemployment_rate.groupby(pd.Grouper(freq="Q")).last()

#Inflation Rate
cpi = fred.get_series('CPIAUCSL')
inflation_rate_p2p = cpi.pct_change()
inflation_rate = cpi.pct_change(periods=12)
inflation_rate_quarterly = inflation_rate.groupby(pd.Grouper(freq="Q")).last()
inflation_rate_quarterly.index = inflation_rate_quarterly.index + timedelta(days=1)

#Population
population = fred.get_series("POPTHM")
population_quarterly = population.groupby(pd.Grouper(freq="Q")).last()
population_quarterly.index = population_quarterly.index + timedelta(days=1)


In [7]:
intro_text = \
"""
Indicators of impending economic and financial crises have consistently impacted the fiscal and monetary policy decisions of countries. 
With the introduction of new financial products, globalization, extraordinary technological advancements,
and the gradual increase of stock market relevance over the past century, the global financial market has drastically changed the
frontier of financial theories. The global financial crisis of 2008 signified the inevitability of financial crises despite advancements; 
now we face a new threat: COVID-19. Within this report, I will identify and explain reliable economic indicators, describe relevant issues
in the context of our current COVID-19 crisis, and provide visualizations for the reader.
"""
display(HTML("<h3 style='text-align:center;'>Introduction<br></h3>\
              <p style='text-align:justify;font-size:16px;'>" + intro_text + "</p>"))

In [8]:
method_text = \
"""
Data was retrieved from The Federal Reserve through this third-party API: {}. All
visualizations were created with the Bokeh library with Python 3. Though the Federal
Reserve was created in 1913, the macroeconomic trends I will discuss were not measured
until after WWII.
""".format("<a target="_blank" href='https://github.com/mortada/fredapi'>https://github.com/mortada/fredapi</a>")
display(HTML("<p style='text-align:center;font-size:20px;'>Methods<br></p>\
              <p style='text-align:justify;font-size:16px;'>" + method_text + "</p>"))

In [26]:
display(HTML("""
<h3 align='center'>
    The Origin and Analysis of Gross Domestic Product
    </h3>
<img style='margin-left:auto;margin-right:auto;display:block;' src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/Simon_Kuznets_1971b.jpg/220px-Simon_Kuznets_1971b.jpg" />
<p{0}>
    <b>In 1934, Dr. Simon Kuznets presented his findings on the National Income to the 73rd U.S. Congress together with the Acting Secretary of Commerce John Dickinson.</b>
    Kuznets was employed as an economist at the National Bureau of Economic Research (NBER) and presented his meticulous research to the U.S. Department of Commerce (USDOC).
    Kuznets and USDOC formalized the findings in “National Income, 1929-32” where the concepts of Gross National Product (GNP) and Gross Domestic Product (GDP) originated.
    <br><br>
    GNP is the total domestic and foreign output claimed by residents of a country, consisting of gross domestic product (GDP), plus factor incomes earned by foreign residents, minus
    income earned in the domestic economy by nonresidents.
    <sup><a target="_blank" href='https://www.investopedia.com/terms/g/gnp.asp'>[1]</a></sup>
    <br><br>
    <b>GNP = GDP + Net Income Inflow from Overseas – Net Income Outflow to Foreign Countries</b>
    <br><br>
    GDP is the total monetary or market value of all the finished goods and services produced within a country's borders in a specific time period. As a broad measure of overall domestic production,
    it functions as a comprehensive scorecard of the country’s economic health.<sup><a target="_blank" href='https://www.investopedia.com/terms/g/gdp.asp'>[2]</a></sup>
    <br><br>
    GDP, often represented by the variable Y, is the sum of Personal Expenditures, Investments, Government Spending, and Net Exports minus Net Imports. The equation is below:
    <br><br>
    <b>Y = C + I + G + NX</b>
    <br><br>
    After the Bretton Woods conference in 1944, GNP was replaced by GDP as the standard means for measuring national economies.
    <br><br>
    Kuznets disapproved of using GDP as a general indication of welfare stating "the welfare of a nation can scarcely be inferred from a measure of national income."<sup><a target="_blank" href='https://fraser.stlouisfed.org/title/national-income-1929-1932-971'>[3]</a></sup>
    <br><br>
    These are the main criticisms of using GDP:
    <ul>
        <li>It doesn't account for unofficial income sources.</li>
        <li>It doesn't account for income repatriation to foreign investors, leading to an overstatement of domestic economic output.</li>
        <li>It emphasizes material output over the populations' well-being.</li>
        <li>It only considers final goods and new capital investment in the production cycle, not business-to-business activity.</li>
    </ul>
    </p>

""".format(" style='text-align:justify;font-size:16px;margin-left:auto;margin-right:auto;'")))

In [10]:
def hover(hover_color="#ffff99"):
            return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])
                
styles = [
        hover(),
        dict(selector="th", props=[("font-size", "100%"),
                                ("text-align", "center"),]),
        dict(selector="td", props=[("text-align", "center"),]),
        dict(selector="caption", props=[("caption-side", "bottom")]),
        dict(selector="table",props=[("margin-left","auto"),
                                    ("margin-right","auto"),
                                    ("background-color","white")])
]

In [26]:
source1 = pd.DataFrame(nominal_gdp*10**3,columns=['gdp'])
hover2 = HoverTool(
    names=['gdp',],
    tooltips=[
        ('date'        ,'@index{%F}'),
#         ('population growth'   ,'@population{%0.3f}'),
        ('GDP','@gdp{$0,0f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)
gdp = figure(sizing_mode = 'scale_width',
             plot_height=325,
             tools=[hover2,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Millions of Dollars",
             title="GDP")
gdp.line(x='index',y='gdp',source=source1,name='gdp',color='green')
gdp.title.align='center'
gdp.yaxis.formatter=NumeralTickFormatter(format="$0,0")

show(gdp)


In [13]:
source1 = pd.DataFrame(nominal_gdp.pct_change(periods=4).dropna(),columns=['gdp'])
hover2 = HoverTool(
    names=['gdp',],
    tooltips=[
        ('date'        ,'@index{%F}'),
#         ('population growth'   ,'@population{%0.3f}'),
        ('GDP growth','@gdp{%0.3f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)
gdpg = figure(sizing_mode = 'scale_width',
             plot_height=325,
             tools=[hover2,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Percent",
             title="Annual GDP Growth Rate")
gdpg.line(x='index',y='gdp',source=source1,name='gdp',color='green')
gdpg.title.align='center'
gdpg.yaxis.formatter=NumeralTickFormatter(format="%0.00")
show(gdpg)

In [14]:
source = pd.DataFrame((nominal_gdp/population_quarterly)*10**6)
source.columns = ['GDP per capita']

# output_file("gdp_per_capita.html")
hover_gdppc = HoverTool(
    names=['gdppc',],
    tooltips=[
        ('date','@index{%F}'),
        ('GDP per capita','@{GDP per capita}{$0,0f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)
gdp_per_cap = figure(sizing_mode = 'scale_width',
                     plot_height=325,
                     tools=['xwheel_zoom',hover_gdppc,'reset'],
                     x_axis_type='datetime',
                     x_range=(source.dropna().index[0],source.index[-1]),
                     x_axis_label="Date",
                     y_axis_label="Dollars",
                     title="GDP per capita")
gdp_per_cap.line(x='index',y='GDP per capita',source=source,color='green',name="gdppc")
gdp_per_cap.title.align='center'

gdp_per_cap.yaxis.formatter=NumeralTickFormatter(format="$0,0")
show(gdp_per_cap)

In [11]:
source = pd.DataFrame(population,columns=['population'])
hover1 = HoverTool(
    names=['pop',],
    tooltips=[
        ('date'        ,'@index{%F}'),
        ('population'   ,'@population{0,0}'),
#         ('GDP growth','@gdp{%0.3f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)

# tools = 'wheel_zoom,xbox_select,reset'

# output_file("population.html")
pop = figure(sizing_mode = 'scale_width',
             plot_height=325,
             tools=[hover1,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Total",
             title="Population")
pop.line(x='index',y='population',name='pop',source=source,color='blue')

pop.title.align='center'
pop.yaxis.formatter=NumeralTickFormatter(format="0,0")
show(pop)

In [12]:
source = pd.DataFrame(population.pct_change(periods=12),columns=['pop_grow'])
hover1 = HoverTool(
    names=['popg',],
    tooltips=[
        ('date'        ,'@index{%F}'),
        ('population growth'   ,'@pop_grow{%0.2f}'),
#         ('GDP growth','@gdp{%0.3f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)

# tools = 'wheel_zoom,xbox_select,reset'

# output_file("population.html")
popg = figure(sizing_mode = 'scale_width',
             plot_height=325,
             tools=[hover1,'xwheel_zoom','reset'],
             y_range=(0,float(source.max())*1.1),
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Percent",
             title="Annual Population Growth Rate")
popg.line(x='index',y='pop_grow',name='popg',source=source,color='blue')

popg.title.align='center'
popg.yaxis.formatter=NumeralTickFormatter(format="%0.00")
show(popg)

display(HTML("""
<p{0}>
    GDP per capita is a relative measure for analyzing economic growth spread over the population. It is used by
    economists alongside GDP to analyze the prosperity of a country.
    The GDP has grown by an average rate of {1}% annually. The population has grown by an average rate of {2}% annually.
    If a country’s GDP per capita is growing with a stable population level it can potentially be the result of technological
    progressions that are producing more output with the same population level. However, if the population is growing at a more
    rapid pace than the GDP, it could mean a gradual decline in the standard of living.
    <br><br>
    Within the context of the COVID-19 pandemic, we could see a rise or fall in population growth, that depends on if 
    households are more likely to produce more children with work-from-home policies and the duration of lockdown orders
    affecting the rate of increase in family units. The GDP will likely see a negative growth rate in excess of the 2008 
    financial crisis for reasons I will explain later.
    </p>
""".format(" style='text-align:justify;font-size:16px;margin-left:auto;margin-right:auto;'",
           round(nominal_gdp.pct_change(periods=4).mean()*100,1),
           round(population.pct_change(periods=12).mean()*100,1)
          )))

In [16]:
source = pd.concat([pd.DataFrame(personal_consumption_expenditures),government_consumption_expenditures,gross_private_domestic_investment,net_exports],join='outer',axis=1)
source.columns = ['consumer spending','government spending','investments','net exports']
source = source*1000

# output_file("gdp_components.html")
hover_gdpc = HoverTool(
    names=['consumer_spending',],
    tooltips=[
        ('date','@index{%F}'),
        ('personal expenditures','@{consumer spending}{$0,0f}'),
        ('government spending','@{government spending}{$0,0f}'),
        ('investments','@investments{$0,0f}'),
        ('net exports','@{net exports}{$0,0f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)
gdpc = figure(sizing_mode = 'scale_width',
             plot_height=325,
             tools=[hover_gdpc,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_range=(source.dropna().index[0],source.index[-1]),
             active_scroll="xwheel_zoom",
             x_axis_label="Date",
             y_axis_label="Millions of Dollars",
             title="GDP Components Over Time")
gdpc.line(x='index',y='consumer spending',source=source,color='blue',name="consumer_spending",legend_label='Personal Expenditures')
gdpc.line(x='index',y='government spending',source=source,color='red',legend_label='Government Spending')
gdpc.line(x='index',y='investments',source=source,color='green',legend_label='Private Investments')
gdpc.line(x='index',y='net exports',source=source,color='black',legend_label='Net Exports')
gdpc.title.align='center'
gdpc.legend.location='top_left'
gdpc.legend.label_text_font_size="10px"
gdpc.legend.border_line_alpha=0.0
gdpc.legend.background_fill_alpha=0.0
gdpc.yaxis.formatter=NumeralTickFormatter(format="$0,0")
display(HTML("""
<h4 align='center'>A Closer Look Into GDP</h4>
<p{0}>
    Here, I have split the components of GDP I described earlier. Two important things to consider in these graphs:
    the major role that consumer spending plays in GDP, and Net Exports can be negative. When Net Exports is negative,
    it indicates the country imports more than it exports, meaning more revenues are going to foreign nations than are
    being accumulated from foreign nations. GDP Component data for the 1st quarter of 2020 has yet to be reported.
    </p>
""".format(" style='text-align:justify;font-size:16px;margin-left:auto;margin-right:auto;'")))
show(gdpc)

In [17]:
source = pd.concat([pd.DataFrame(personal_consumption_expenditures/nominal_gdp),
                    government_consumption_expenditures/nominal_gdp,
                    gross_private_domestic_investment/nominal_gdp,
                    net_exports/nominal_gdp],
                   join='outer',axis=1)
source.columns = ['personal expenditures','government spending','investments','net exports']

# output_file("gdp_components_percent.html")
hover_gdpp = HoverTool(
    names=['consumer_spending',],
    tooltips=[
        ('date','@index{%F}'),
        ('personal expenditures','@{personal expenditures}{%0f}'),
        ('government spending','@{government spending}{%0f}'),
        ('investments','@investments{%0f}'),
        ('net exports','@{net exports}{%0f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)
gdpp = figure(sizing_mode = 'scale_width',
             plot_height=325,
             tools=[hover_gdpp,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_range=(source.dropna().index[0],source.index[-1]),
             active_scroll="xwheel_zoom",
             x_axis_label="Date",
             y_axis_label="Percent",
             title="GDP Components as a % of GDP")
gdpp.line(x='index',y='personal expenditures',source=source,color='blue',name="consumer_spending",legend_label='Personal Expenditures')
gdpp.line(x='index',y='government spending',color='red',source=source,legend_label='Government Spending')
gdpp.line(x='index',y='investments',color='green',source=source,legend_label='Private Investments')
gdpp.line(x='index',y='net exports',color='black',source=source,legend_label='Net Exports')
gdpp.title.align='center'
gdpp.legend.location='top_left'
gdpp.legend.label_text_font_size="10px"
gdpp.legend.border_line_alpha=0.0
gdpp.legend.background_fill_alpha=0.0
gdpp.yaxis.formatter=NumeralTickFormatter(format="%0")

show(gdpp)
source = source*100
display(HTML("""
<p>
With these graphs, my goal is to depict that the components of GDP remain relatively stationary as a portion of GDP. Statistics for each component as a percentage of GDP is below:
</p>
<div style='display: flex;align-self:center;justify-content:center;'><br>""" + \
                source.describe().drop('count',axis=0).T.style.set_table_styles(styles)\
                    .set_caption("Hover to highlight.")\
                    .format("{:.2f}%")\
                    .render()\
             +"""</div>
<p{0}>
    Private Investments reached an all time low of just under 13% of GDP during the 2008 financial crisis. Net Exports
    and Government Spending actually increased in response. You will see in the next section, that the Federal Government 
    printed more money to mitigate the effect of the rapidly diminishing corporate investments. 
    <br><br>
    We should expect to see is a dramatic decrease in Personal Expenditures and an increase in Government Spending with COVID-19. 
    Households simply cannot afford the same level of consumption on luxuries as before. There will be a shock to demand for necessities such as food manufacturing, healthcare, IT-services, etc., but luxury goods and
    services providers are not essential businesses in the first place. If a luxury provider were able to attain an essential
    business license, the demand for their product has still dropped off significantly. 
    <br><br>
    The Department of Homeland Security has released
    a memo detailing their guidance to state, local, tribal, and territorial jurisdictions and the private sector on
    defining essential critical infrastructure workers.
    <sup><a target="_blank" href='https://www.cisa.gov/sites/default/files/publications/Version_3.0_CISA_Guidance_on_Essential_Critical_Infrastructure_Workers_4.pdf'>[4]</a></sup>
     Homeland Security maintains that their guidance is merely advisory. In the memo, under the section "CONSIDERATIONS
    FOR GOVERNMENT AND BUSINESS" it states "Everyone should follow guidance from the Centers for Disease Control and Prevention (CDC), as well as state
    and local government officials, regarding strategies to limit disease spread."
    </p>
             """.format(" style='text-align:justify;font-size:16px;margin-left:auto;margin-right:auto;'")
))

,mean,std,min,25%,50%,75%,max
personal expenditures,63.39%,3.12%,58.56%,60.36%,63.32%,66.79%,68.66%
government spending,20.56%,2.17%,15.22%,18.97%,20.65%,22.36%,25.07%
investments,17.29%,1.63%,12.77%,16.16%,17.35%,18.31%,21.39%
net exports,-1.24%,2.00%,-5.90%,-2.90%,-0.93%,0.25%,4.72%


In [18]:
display(HTML("""
<h3 align='center'>
    The Quantity Theory of Money
    </h3>
<img style='margin-left:auto;margin-right:auto;display:block;' src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/7d/Irvingfisher.jpg/220px-Irvingfisher.jpg" />
<p{0}>
    <b>Dr. Irving Fisher formulated the Quantity Theory of Money in terms of the "Equation of Exchange" in 1910.</b>
    We can credit Fisher with the introduction of econometrics, the application of statistical methods and mathematical models to economic observations.
    He wrote many books and articles on his theories of interest rates, capital, the concept of inflation, and price indexes. Fisher earned the first Ph.D. in economics ever awarded by Yale.
    <br><br>
    <b>MV + M'V' = PT</b>
    </p>
<div style='margin-left:auto;margin-right:auto;text-align:center;'>
    <p{0}>
        The purchasing power of money, as theorized by Fisher, is determined by five factors:
        </p>
    <ol style='display:inline-block;text-align:left;'>
        <li><b>M</b>: The volume of money in circulation.</li>
        <li><b>V</b>: The velocity of money circulation in a given year.</li>
        <li><b>M'</b>: The volume of bank deposits subject to check.</li>
        <li><b>V'</b>: Their velocity of circulation.</li>
        <li><b>PT</b>: The general level of trade accounting for inflation.</li>
        </ol>
    </div>
<p{0}>
        <b>T</b>, or the level of trade, is substituted by <b>Y'</b>; the real GDP.
        <b>P</b>, or the GDP price deflator, is a measure of the level of prices of all new, domestically produced, final goods and services in an economy.
        <b>P * Y'</b> is by definition equal to the nominal GDP; the inflation-<u>un</u>adjusted GDP.
        The terms <b>M</b> and <b>M'</b> are combined in the M2 monetary supply measure from the Federal Reserve, as are the terms <b>V</b> and <b>V'</b>
        in the M2V velocity of M2 circulation measure. Given these facts, we derive the equation:
        <br>
        <b>MV = PY' = Y</b>
        <br>
        <b>Money Supply * Velocity of Money = GDP Price Deflator * Real GDP = Nominal GDP</b>
        <br><br>
        It is important to realize that an increase in the supply of money will inevitably lead to a decrease in the value of money, but in the short term, 
        the generation of money is used to combat unemployment and raise the GDP. This method is what is known as Monetarism: the view that controlling
        changes to money supply is the sole determinant of economic growth.<sup><a target="_blank" href='https://www.investopedia.com/terms/m/monetarism.asp'>[5]</a></sup>
    </p>
    
""".format(" style='text-align:justify;font-size:16px;margin-left:auto;margin-right:auto;'"," style='text-align:center;font-size:16px;margin-left:auto;margin-right:auto;'")))

In [24]:
source = pd.DataFrame((m2_money_supply*10**2).pct_change(periods=12),columns=['money supply'])
source1 = pd.DataFrame(m2_money_velocity.pct_change().loc[m2_money_velocity.index >= m2_money_supply.index.min()],columns=['money velocity'])

hover = HoverTool(
    names=['money',],
    tooltips=[
        ('date'        ,'@index{%F}'),
        ('money supply'   ,'@{money supply}{%0.2f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
    ,line_policy="nearest"
)
hover1 = HoverTool(
    names=['velocity',],
    tooltips=[
        ('date'        ,'@index{%F}'),
        ('velocity'   ,'@{money velocity}{%0.2f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
    ,line_policy="interp"
)


m_s = figure(sizing_mode = 'scale_width',
           plot_height=350, 
           tools=[hover,hover1,'xwheel_zoom','reset'],
           x_axis_type="datetime", 
           title="Quarterly Change in Money Supply",
           x_axis_label="Date",
           y_axis_label="Percent")#active_drag="xbox_select")
m_s.line(x='index',
         y='money supply',
         source=source,
         color='green',
         name='money')
m_s.line(x='index',
         y='money velocity',
         source=source1,
         color='blue',
         name='velocity')
m_s.title.align='center'
m_s.yaxis.formatter = NumeralTickFormatter(format="%0.0")

display(HTML("""
<p{0}>
    With the following graph, you can view the percent change in the money supply from 12 weeks/1 quarter prior.
    You can see a few spikes in the money supply:
    </p>
    <ul>
        <li>February 1983 - 6%</li>
        <li>September 2001 - over 5%</li>
        <li>December 2008 - around 4.5%</li>
        <li>August 2011 - over 5%</li>
        <li>April 2020 - 13.5%</li>
        </ul>
""".format(" style='text-align:justify;font-size:16px;margin-left:auto;margin-right:auto;'")))
show(m_s)
display(HTML("""
<p{0}>
    If you are unaware of the events coinciding with the expansionary money supply periods, they are: 
    the introduction of monetary policy after the 1980-1982 recession,
    economic uncertainty associated with the 9-11 attacks,
    the financial meltdown of the real estate bubble in 2008,
    the stock market crash after Standard & Poor's credit rating downgrade of U.S. sovereign debt from AAA to AA+,
    and COVID-19, respectively.
    <br><br>
    For both the 2008 and 2011 downturns, economic stimulus acts were passed to spur the growth of the economy.
    There are a few categories these legislative tools target: expanding and extending unemployment benefits,
    tax cuts or stimulus checks for individuals, tax credits and incentives for businesses, and infrastructure funding. 
    <br><br>
    There is also a contractionary trend of the money velocity accompanying the positive changes in money supply, counteracting positive changes in the GDP.
    This phenomenon of a negative relationship between the supply and velocity of money is part of the Ricardian Equivalence.
    The Ricardian Equivalence proposes individuals who receive extra money will save it in order to pay for future
    tax increases.
    <sup><a target="_blank" href='https://www.investopedia.com/terms/r/ricardianequivalence.asp'>[6]</a></sup>
    According to a survey conducted by Reuters and the University of Michigan, 25% of stimulus check recipients intended to
    use the money to rebuild savings and 45% intended to use the money to pay down debt.
    <sup><a target="_blank" href='https://www.reuters.com/article/us-usa-rebate-umich/only-30-pct-of-bush-tax-rebates-to-be-spent-survey-idUSN0251441720080502'>[7]</a></sup>
    Economic stimuli rely on the affected parties to spend their boosted income, otherwise the result will necessarily 
    be an increase in inflation.
    </p>
""".format(" style='text-align:justify;font-size:16px;margin-left:auto;margin-right:auto;'")))

In [20]:
source = pd.concat([pd.DataFrame(unemployment_rate),inflation_rate],join='outer',axis=1)
source.columns = ['unemployment','inflation']

from scipy.signal import find_peaks, peak_prominences
peaks = pd.DataFrame(source.unemployment.iloc[find_peaks(source.unemployment,prominence=0.01,distance=12)[0]])
peaks['val'] = 'peak'
troughs = pd.DataFrame(source.unemployment.iloc[find_peaks(source.unemployment.apply(lambda x: x*-1),prominence=0.01,distance=12)[0]])
troughs['val'] = 'trough'
peaks_and_troughs = pd.concat([peaks.reset_index(),troughs.reset_index()],axis=0)

peaks_and_troughs = peaks_and_troughs.sort_values('index').reset_index()
peaks_and_troughs['time_between'] = peaks_and_troughs['index'].diff()
peaks_and_troughs['unemployment_adj'] = peaks_and_troughs['unemployment'] * 100

del peaks_and_troughs['level_0']

average_time_peak_to_trough = str(peaks_and_troughs.loc[peaks_and_troughs['val']=='trough','time_between'].mean().days)
average_time_trough_to_peak = str(peaks_and_troughs.loc[peaks_and_troughs['val']=='peak','time_between'].mean().days)

In [21]:
hover = HoverTool(
    names=['unem',],
    tooltips=[
        ('date'        ,'@index{%F}'),
        ('inflation'   ,'@inflation{%0.2f}'),
        ('unemployment','@unemployment{%0.2f}')
    ],
    formatters={
        '@index':'datetime',
        
    }
    ,mode='vline'
)
tools = 'wheel_zoom,xbox_select,reset'

# output_file("inflation_vs_unemployment.html")
InfVsUnem = figure(sizing_mode = 'scale_width',
             plot_height=350,
             tools=[hover,'xwheel_zoom','reset'],
             x_axis_type='datetime',
             x_axis_label="Date",
             y_axis_label="Percent",
             title="Unemployment and Inflation")
InfVsUnem.line(x='index',y='unemployment',legend_label='Unemployment',name='unem',source=source,color='blue')
InfVsUnem.line(x='index',y='inflation',legend_label='Inflation',source=source, color='red')
InfVsUnem.circle(x=peaks_and_troughs['index'],y=peaks_and_troughs['unemployment'],color='black',size=7)

InfVsUnem.title.align='center'
InfVsUnem.legend.location='top_right'
InfVsUnem.legend.label_text_font_size = "10px"
InfVsUnem.legend.border_line_alpha=0.0
InfVsUnem.legend.background_fill_alpha=0.0
InfVsUnem.yaxis.formatter=NumeralTickFormatter(format="0%")
display(HTML("""
<h3 align='center'>The Dynamics of Unemployment and Inflation</h3>
<p{0}>
    Often shown together are the unemployment rate and inflation rate, due to what's known as the Phillips Curve positing 
    an inverse relationship between the two indicators.
    <sup>
        <a target="_blank" href='https://courses.lumenlearning.com/boundless-economics/chapter/the-relationship-between-inflation-and-unemployment/'>
            [more info]
            </a>
        </sup>
    <br><br>
    Here I have used the annual change in the seasonally-adjusted Consumer Price Index (CPI),
    to indicate the level of inflation. The CPI is a measure of the average change over time in the prices paid by urban
    consumers for major groups of consumer goods and services. Here, I have used prices from 1982-1984 as the base of
    my index, as is standard for the Federal Reserve and the Bureau of Labor Statistics.
    <sup><a target="_blank" href='https://www.bls.gov/opub/btn/volume-1/writing-an-escalation-contract-using-the-consumer-price-index.htm'>[8]</a></sup>
    <br><br>
    Price indexes for the major groups of consumer goods and services are available here: 
    <a target="_blank" href='https://data.bls.gov/PDQWeb/cu'>https://data.bls.gov/PDQWeb/cu</a>.
    <br><br>
    Since stagflation, a period of high inflation and high unemployment, in the mid-1970s and the recession in the early 1980s,
    inflation has mostly remained stationary in the 0-5% range annually. The unemployment rate, however, abruptly rises and slowly contracts; 
    this is depicted in the table following. As of April 1st, 2020, the unemployment rate shows a staggering 14.7%. This level of unemployment
    has not occured since the 1930's with the Great Depression; the population is also 150% higher.
    </p>
""".format(" style='text-align:justify;font-size:16px;margin-left:auto;margin-right:auto;'")))

show(InfVsUnem)

In [22]:
peaks_and_troughs.columns = ['Date','Unemployment Rate','Value','Time Between','Unemployment']
peaks_and_troughs['Date'] = peaks_and_troughs['Date'].apply(lambda x: x.strftime('%A, %b-%d-%Y'))

display(HTML("""<h4 align='center'>Peaks and Troughs of Unemployment</h4>\
     <div display: table;>\
         <div style='display: flex;justify-content:center;flex-direction:column;'>\
             <div style='display: flex-item;align-self:center;'>\
                    Average Time Trough to Peak: """ + average_time_trough_to_peak + """ days<br><br>Average Time Peak to Trough: """+ average_time_peak_to_trough +""" days\
                    </div>\
             <div style='display: flex-item;align-self:center;'><br>""" + \
                peaks_and_troughs.drop('Unemployment Rate',axis=1)[['Date','Unemployment','Time Between','Value']]\
                    .style.set_table_styles(styles)\
                    .set_caption("Hover to highlight.")\
                    .format({"Unemployment":"{:.1f}%"})\
                    .hide_index()\
                    .render()+"""\
             </div>\
             </div>\
         </div>\
<p{0}>\
    The average time from trough to peak shows it takes over 2 years from the lowest level of unemployment to the highest level of unemployment in a 12-month span. \
    The average time from peak to trough is an indicator of economic recovery. It takes an average of 5 years to recover. From 1931 to 1940, the U.S. unemployment \
    rate exceeded 14.5%, the level we are seeing with COVID-19 as of April 1st, 2020. When I spoke about the GDP per capita before, I depicted that the growth of GDP in excess \
    of the growth of the population could be due to techological progress. Companies may more readily invest in technological processes than hire employees for many reasons including: </p>\
    <ul>\
        <li>The possible spread of COVID-19 to the employers or their employees</li>\
        <li>The economic gain of automation</li>\
        <li>The availability of automation and AI</li>\
    </ul>
<p{0}>
    They're compelling reasons. Automation alone has caused what's called frictional unemployment throughout this past century, but with Artificial Intelligence, companies can more easily 
    replace primarily human-based jobs.
    </p>""".format(" style='text-align:justify;font-size:16px;margin-left:auto;margin-right:auto;'")))

Date,Unemployment,Time Between,Value
"Saturday, Oct-01-1949",7.9%,NaT,peak
"Friday, May-01-1953",2.5%,1308 days 00:00:00,trough
"Wednesday, Sep-01-1954",6.1%,488 days 00:00:00,peak
"Friday, Mar-01-1957",3.7%,912 days 00:00:00,trough
"Tuesday, Jul-01-1958",7.5%,487 days 00:00:00,peak
"Monday, Feb-01-1960",4.8%,580 days 00:00:00,trough
"Monday, May-01-1961",7.1%,455 days 00:00:00,peak
"Wednesday, Jan-01-1969",3.4%,2802 days 00:00:00,trough
"Tuesday, Dec-01-1970",6.1%,699 days 00:00:00,peak
"Monday, Oct-01-1973",4.6%,1035 days 00:00:00,trough


In [23]:
# display(HTML("""
# <h3 align='center'>Conclusion</h3>
# <p{0}>
#     We are facing a level of economic and personal uncertainty unlike ever before. 
#     It is unclear how high the unemployment rate will rise or how long it will last.
#     This historic rise in the money supply is being used as a countermeasure to the
#     impending fall in personal expenditures and private investments. 
#     </p>
# """))

In [35]:
display(HTML("""
<h3 align='center'>Citations</h3>
<p>
    [1] Chappelow, J. (March 2020) Gross National Product (GNP). Investopedia.
    Retrieved from: 
    <a target="_blank" href='https://www.investopedia.com/terms/g/gnp.asp'>
        https://www.investopedia.com/terms/g/gnp.asp
        </a>
    </p>
<p>
    [2] Chappelow, J. (April 2020) Gross Domestic Product—GDP. Investopedia.
    Retrieved from: 
    <a target="_blank" href='https://www.investopedia.com/terms/g/gdp.asp'>
        https://www.investopedia.com/terms/g/gdp.asp
        </a>
    </p>
<p>
    [3] U.S. Bureau of Foreign and Domestic Commerce, Seventy-Third Congress, 1933-1935, & Kuznets, S. 
    National Income, 1929-1932. Washington: U.S. Government Printing Office, 1934.
    Retrieved from: 
    <a target="_blank" href='https://fraser.stlouisfed.org/title/national-income-1929-1932-971'>
        https://fraser.stlouisfed.org/title/national-income-1929-1932-971
        </a>
    </p>
<p>
    [4] Director of the Cybersecurity and Infrastructure Security Agency. (April 17, 2020). Guidance on the Essential Critical Infrastructure Workforce: Ensuring Community and National Resilience in COVID-19 Response. Version 3.0. U.S. Department of Homeland Security Cybersecurity & Infrastructure Security Agency. Retrieved from: 
    <a target="_blank" href='https://www.cisa.gov/sites/default/files/publications/Version_3.0_CISA_Guidance_on_Essential_Critical_Infrastructure_Workers_4.pdf'>
        https://www.cisa.gov/sites/default/files/publications/Version_3.0_CISA_Guidance_on_Essential_Critical_Infrastructure_Workers_4.pdf
        </a>
    </p>
<p>
    [5] Ganti, A. (November 2019). Monetarism. Investopedia.
    Received from: 
    <a target="_blank" href='https://www.investopedia.com/terms/m/monetarism.asp'>
        https://www.investopedia.com/terms/m/monetarism.asp
        </a>
    </p>
<p>
    [6] Kenton, W. (November 2019). Ricardian Equivalence. Investopedia.
    Retrieved from: 
    <a target="_blank" href='https://www.investopedia.com/terms/r/ricardianequivalence.asp'>
        https://www.investopedia.com/terms/r/ricardianequivalence.asp
        </a>
    </p>
<p>
    [7] Rodrigues, V. (May 2008). "Only 30 pct of Bush tax rebates to be spent: survey". Reuters. Retrieved from
    <a target="_blank" href='https://www.reuters.com/article/us-usa-rebate-umich/only-30-pct-of-bush-tax-rebates-to-be-spent-survey-idUSN0251441720080502'>
        https://www.reuters.com/article/us-usa-rebate-umich/only-30-pct-of-bush-tax-rebates-to-be-spent-survey-idUSN0251441720080502
        </a>
    </p>
<p>
    [8] Crawford, M., & Stewart, K. (November 2012). Consumer Price Index program.
    “Writing an escalation contract using the Consumer Price Index,” Beyond
    the Numbers: Prices & Spending. vol. 1, no. 19. U.S. Bureau of Labor Statistics. Retrieved from
    <a target="_blank" href='https://www.bls.gov/opub/btn/volume-1/writing-an-escalation-contract-using-the-consumer-price-index.htm'>
        https://www.bls.gov/opub/btn/volume-1/writing-an-escalation-contract-using-the-consumer-price-index.htm
        </a>
    </p>

"""))